In [1]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def ARIMA(historical_data):
    model = auto_arima(historical_data, seasonal=True, stepwise=True, m=6, method="bfgs", maxiter=20, suppress_warnings=True) #Auto-Arima passt automatisch p un q an
    forecast = model.predict(n_periods=10)
    forecast_df = pd.DataFrame(forecast, columns=["Predicted Price"])
    fitted_values = model.fittedvalues()
    order = model.order  # (p, d, q)
    
    return forecast_df, fitted_values, order

In [6]:
data = yf.download("MDO.DE", period = "max")
data.drop(columns = ["Open", "High", "Low", "Volume", "Adj Close"], axis = 1, inplace = True)
end_zeitpunkte = ["2023-07-07"]
end_zeitpunkte_ts = [datetime.strptime(end_zeitpunkte[0], "%Y-%m-%d").date()]

#hier Zeitraum der historischen Daten anpassen:
time_horizont = 3
start_zeitpunkte_ts = [end_zeitpunkte_ts[0]-relativedelta(years=time_horizont)]
filtered_df = data.loc[start_zeitpunkte_ts[0]:end_zeitpunkte_ts[0]]
hist_data = filtered_df[: len(filtered_df)-10]
hist_data_for_function = hist_data.copy()
prog_data = filtered_df[len(filtered_df)-10 :]
index_list = prog_data.index.tolist()

forecast_df, fitted_values, order = ARIMA(hist_data_for_function)

forecast_df.set_index(pd.Index(index_list), inplace=True)
result = pd.concat([prog_data, forecast_df], axis=1)

[*********************100%***********************]  1 of 1 completed


C:\Users\pasca\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [10]:
fig2 = px.line(template = "simple_white")
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Close"], mode = "lines", name = "Tatsächlicher Kurs", line_color = "red"))
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Predicted Price"], mode = "lines", name = "Prognose", line_color = "blue"))
fig2.update_layout(xaxis_title = "Datum", yaxis_title = "MC Donalds Kurspreis in EUR", xaxis= {"type": "category"}, legend=dict(x=0, y=1))
fig2.write_image("Prog_Arima.pdf")

In [18]:
fig3 = px.line(template = "simple_white")
fig3.add_trace(go.Scatter(x = hist_data.index[629:], y = hist_data["Close"][629:], mode = "lines", name = "Tatsächliche Daten", line_color = "red"))
fig3.add_trace(go.Scatter(x = hist_data.index[629:], y = fitted_values[629:], mode = "lines", name = "Prognostizierte Daten", line_color = "blue"))
fig3.update_layout(xaxis_title = "Datum", yaxis_title = "MC Donalds Kurspreis in EUR", legend=dict(x=0, y=1))
fig3.write_image("Train_Arima.pdf")